In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

In [3]:
train_dataset = MNIST(True)
test_dataset = MNIST(False)

In [4]:
train_x, train_y = train_dataset.train_data.numpy(), train_dataset.train_labels
test_x, test_y = test_dataset.test_data.numpy(), test_dataset.test_labels
train_y2 = to_one_hot(train_y, 10).numpy()
test_y2 = to_one_hot(test_y, 10).numpy()
train_y = train_y.numpy()
test_y = test_y.numpy()

In [5]:
train_data = FromNumpy(train_x[:, None, :, :], [train_y2, train_y], batch_size=256)
test_data = FromNumpy(test_x[:, None, :, :], [test_y2, test_y], batch_size=256)

In [6]:
class Auxiliary(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        self.store = None
    
    def forward(self, x):
        # Store the result
        # Return the origin value
        self.store = self.f(x)
        return x 

In [11]:
model = builder([
    nn.Conv2d(1, 20, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(20, 50, 5, 1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    Flatten(),
    nn.Linear(4*4*50, 500),
    nn.ReLU(),
    Auxiliary(builder([
        nn.Linear(500, 10),
        nn.Sigmoid()
    ])),
    nn.Linear(500, 10),
    nn.Softmax(1)
])

In [12]:
runner = Runner(model, train_data, test_data, 'adam', ['bce', 'll'])

In [13]:
runner.fit(4)

  Epochs    train_loss    train_acc      val_loss      val_acc    
    1*        0.5010        0.9154        0.0939        0.9752    


    2         0.0529        0.9875        0.0392        0.9906    


    3         0.0239        0.9949        0.0542        0.9853    


    4         0.0201        0.9951        0.0295        0.9920    


In [14]:
model = runner.model

In [17]:
data = next(iter(train_data))
x = data[0][0]
x = x[None, :, :, :]
x = torch.rand_like(x) + x
out1 = model(x.cuda())
out2 = get_aux_out(model)[0]
print(out1)
print(out2)
print(out1*out2)


tensor([[9.9997e-01, 2.5740e-11, 1.5347e-09, 4.9906e-10, 7.6908e-10, 5.0669e-06,
         6.5643e-06, 4.7264e-08, 1.4733e-05, 1.6368e-07]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
tensor([[9.9994e-01, 5.0191e-09, 8.2649e-08, 1.0085e-07, 1.3195e-06, 9.1732e-04,
         8.4961e-05, 2.0007e-05, 5.2035e-03, 3.8097e-07]], device='cuda:0',
       grad_fn=<SigmoidBackward>)
tensor([[9.9991e-01, 1.2919e-19, 1.2684e-16, 5.0329e-17, 1.0148e-15, 4.6480e-09,
         5.5771e-10, 9.4562e-13, 7.6664e-08, 6.2356e-14]], device='cuda:0',
       grad_fn=<MulBackward0>)
